In [ ]:
%matplotlib inline
import random
from images import *
from NumberClass import *
from math import *
from operator import itemgetter, attrgetter
import numpy as np
import matplotlib.pyplot as plt
from numpy import ma
from matplotlib import colors, ticker, cm
from matplotlib.mlab import bivariate_normal

In [ ]:
''' Part 1: Training

'''
images = imageArray('trainingimages','traininglabels')
#print len(images)
numberClass = []
labelArray = []
x = []
y = []
maxIndex = 0

for i in range(10) :
    numberClass.append(NumberClass()) 
    labelArray.append(0)

'''Adding features to the number class'''
for i in range(5000):
    numberValue = int(images.getLabels(i))
    imageData = []
    imageData = images.getImage(i)
    numberClass[numberValue].addToCount(1)
    for row in range(28):
        for column in range(28): 
            if imageData[row][column] == ' ':
                numberClass[numberValue].addToFeature(0,row,column)
            else:
                numberClass[numberValue].addToFeature(1,row,column)
print ('features done')

'''calculating the weight vector as function of epoch'''

for t in range(21):
    correct = 0
    for i in range(5000):
        numberValue = int(images.getLabels(i))
        dotArray = []
        if numberClass[numberValue].getStatus() == 0:
            maxIndex = numberValue
            numberClass[numberValue].changeStatus()
            #print maxIndex
        else:
            for index in range (10):
                dotArray.append(ma.dot(numberClass[index].getWeights(), numberClass[index].getFeature()))
            maxIndex = dotArray.index(max(dotArray))
            #print maxIndex
            #print ('printing '),maxIndex
    
        if maxIndex != numberValue:
            for row in range(28):
                for column in range(28): 
                    numberClass[numberValue].updateWeightVector((float(5000)/(1000+2*t)),row,column)
                    numberClass[maxIndex].updateWeightVector(-(float(5000)/(1000+2*t)),row,column)
        else:
            correct += 1
    print correct
x.append(0)
#508
#for index in range(10):
    #print index,(' '),numberClass[index].getWeights(), ('\n')

features done
511